In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [1]:
def derivtransfer(FPs,type='tanhp'):
    xe,xi=FPs[0],FPs[1]
    if type=='tanhp':
        derive=1/np.cosh(xe)**2
        derivi=1/np.cosh(xi)**2
        return np.array([derive,derivi])
def FPfunc(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=np.tanh(x0+x1)+1
        resp1=np.tanh(x0-x1)+1
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]  
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        resp0=(np.tanh(x0+x1))
        resp1=(np.tanh(x0-x1))
        return [
            x0-JE*resp0+JI*resp1,
            x1+a*resp0-b*resp1
        ]        
def FPfuncSVD(x,*data):
    JE,JI,a,b,tfunc=data
    if tfunc=='tanhp':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=np.tanh(x0*m[0,0]+x1*m[0,1])+1
        resp1=np.tanh(x0*m[1,0]+x1*m[1,1])+1
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]
    elif tfunc=='tanh':
        x0 = float(x[0])
        x1 = float(x[1])
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+b,Jt[1,0]+a,Jt[1,1]-b
        lvec,sv,rvech=la.svd(Jt)
        m,n=lvec,rvech.T
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        resp0=(np.tanh(x0*m[0,0]+x1*m[0,1])+0.0)
        resp1=(np.tanh(x0*m[1,0]+x1*m[1,1])+0.0)
        return [
            x0-n[0,0]*resp0-n[1,0]*resp1,
            x1-n[0,1]*resp0-n[1,1]*resp1
        ]

In [ ]:
# bseries= np.logspace(-2,  2.0,num =20,base=2)
JE,JI,a,b=1.6,0.8,0.10,0.10
bseries = np.linspace(-15.0,  15.0,num =500)  
nlen=len(bseries)
M=np.array([[1,1],[1,-1]])
xFPseries = np.zeros((nlen,2,2))
svdvalues = np.zeros((nlen,2))
svdvec = np.zeros((nlen,2,4))
kappaMN,kappamnSVD=np.zeros((nlen,2)),np.zeros((nlen,2))
for idxje,JE in enumerate(jeseries):
    for idxb,bv in enumerate(bseries):
        bv*=a
        # calculate SVD
        Jt = np.zeros((2,2))
        Jt[:,0],Jt[:,1]=JE,-JI
        Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=Jt[0,0]-a,Jt[0,1]+bv,Jt[1,0]+a,Jt[1,1]-bv
        if (np.min(Jt[:,0])<0.0) or (np.max(Jt[:,1])>0.0):
            xFPseries[idxje,idxb,:,:]=np.nan
            kappaMN[idxje,idxb,:]=np.nan
            kappamnSVD[idxje,idxb,:]=np.nan
            continue   
        lvec,sv,rvech=la.svd(Jt)
        svdvalues[idxje,idxb,:]=sv
        m,n=lvec,rvech.T
              
        m[:,0]*=sv[0]
        m[:,1]*=sv[1]
        if n[0,0]<0:
            n[:,0]*=(-1)   
            m[:,0]*=(-1)
        if n[0,1]<0:
            n[:,1]*=(-1)   
            m[:,1]*=(-1)
        svdvec[idxje,idxb,:,:2]=m
        svdvec[idxje,idxb,:,2:]=n

        N=np.array([[JE,-a],[-JI,bv]])
        data=(JE,JI,a,bv,'tanh')
        x0=[0.10,0.10]
        results = fsolve(FPfunc,x0,data)
        kappaMN[idxje,idxb,:]=results
        xFP= M@np.reshape(results,(2,1))
        xFPseries[idxje,idxb,0,:]=xFP[:,0]
        
        resultSVD= fsolve(FPfuncSVD,x0,data)
        kappamnSVD[idxje,idxb,:]=resultSVD
        xFPSVD= m@np.reshape(resultSVD,(2,1))
        xFPseries[idxje,idxb,1,:]=xFPSVD[:,0]
        # check eig
        Xderive=derivtransfer(xFPSVD,type='tanh')
        Jcheck=Jt.copy()
        Jcheck[:,0]*=Xderive[0]
        Jcheck[:,1]*=Xderive[1]
        eigv,eigvec=la.eig(Jcheck)
        if eigv[0]>1.0:
            xFPseries[idxje,idxb,:,:]=np.nan
            kappaMN[idxje,idxb,:]=np.nan
            kappamnSVD[idxje,idxb,:]=np.nan
            continue
        # print('Fixed points A:',xFP,' SVD(B):',xFPSVD)
        # overlap under MNT
        diagdphi=np.zeros((2,2))
        for i in range(2):
            diagdphi[i,i]=1/np.cosh(xFPSVD[i])**2
        stabilityMN=N.T@diagdphi@M#
        stabilitymn=n.T@diagdphi@m#
        eigvMN,eigvecMN=la.eig(stabilityMN)
        eigvmn,eigvecmn=la.eig(stabilitymn)
        # print('eigenvalues of overlap MN and mn:',eigvMN,' and ',eigvmn)
